# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [26]:
# !!!!!!!!! IMPORTANT !!!!!!!!!
# In exercises 2, 5 and 19 there are two different options for variable "filename".
# Use the the one with "src/" in the beginning of the word for unit tests and the
# one without it for getting the outputs. Other exercises depend on these ones after
# might need to rerun the whole page.

from collections import defaultdict
from itertools import product
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import re
import math
from numpy.random import choice

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```). 


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [27]:

def dna_to_rna(s):
    transform = {"A":"A", "C":"C", "G":"G", "T":"U"}
    return "".join(transform[nucleo] for nucleo in s)
    
if __name__ == '__main__':
    print(dna_to_rna("AACGTGATTTC"))

AACGUGAUUUC


### Idea of solution

I created a dictionary with mappings from the dna-nucleotides to rna-nucleotides. The only nucleotide that needed to be transformed was thymine to uracil (T -> U), all the others stayed the same.

### Discussion

The code run as I expected, transforming "thymines" in the input to "uracils" in the output.

Output:
"AACGUGAUUUC"

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [28]:
def get_dict():
    filename = "codon_data.html" # use to get the outputs
    # filename = "src/codon_data.html" # use for local unit tests
    
    with open(filename) as f:
        soup = BeautifulSoup(f)
        
    # Clean the data
    data = str(soup.pre).replace("<pre>", "").replace("</pre>", "")
    
    # Find all the triplet-amino acid pairs by using regular expressions
    pattern = r"(\w{3})\s([A-Z*])"
    pairs = re.findall(pattern, data)
    
    # Create the dataframe
    cols = ["triplet", "amino acid"]
    df = pd.DataFrame(pairs, columns=cols)
    
    # Get the columns from the dataframe and squeeze them into series
    triplets = df.triplet.squeeze()
    amino_acids = df["amino acid"].squeeze()
    
    # Set triplets as the index for the amino_acids
    amino_acids.index = triplets
    
    # Turn the series into a dictionary
    return amino_acids.to_dict()
    
if __name__ == '__main__':
    codon_to_aa = get_dict()
    print(codon_to_aa)

{'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}


### Idea of solution

First I used BeautifulSoup to get the table into string format and then I cleaned the data. I used regular expressions to find all the triplet-amino_acid pairs. After this, I turned these pairs into a dataframe and then to seperate series. I then formed the dictionary by setting triplets as the index for amino_acids and then using the "to_dict" method on that series.

### Discussion

Execution went as expected, outputting the dictionary with all the triplet-->amino_acid pairs from the original table.

Output:
{'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}

3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [29]:

def get_dict_list():
    # Get the dict from previous exercise
    og_dict = get_dict()
    
    # Unpack to keys and values
    keys, values = og_dict.keys(), og_dict.values()
    
    # Build the new dictionary
    new_dict = defaultdict(list)
    for k, v in zip(values, keys):
        new_dict[k].append(v)
        
    return dict(new_dict)
    
if __name__ == '__main__':
    aa_to_codons = get_dict_list()
    print(aa_to_codons)

{'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UGA', 'UAG'], 'W': ['UGG'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']}


### Idea of solution

I called the function "get_dict" to get the dictionary from previous exercise. I extracted the keys and values from the dictionary and then zipped them back together in reverse order. After this I build the new dictionary by using defaultdict with lists and appending the values to the lists.

### Discussion

Function worked correctly, mapping every aminoacid to its respective nucleotide triplets.

{'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UGA', 'UAG'], 'W': ['UGG'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']}

With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence. 
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [30]:

def rna_to_prot(s):
    # Get the dictionary that converts triplets to amino acids
    dictionary = get_dict()
    
    # Transform triplets into amino acids and add them to the protein string
    i = 0
    end = len(s) - 3
    prot = ""
    while i <= end:
        codon = s[i:i+3]
        prot += dictionary[codon]
        i += 3
    
    # Turn the list into a string
    return prot

def dna_to_prot(s):
    return rna_to_prot(dna_to_rna(s))

if __name__ == '__main__':
    print(dna_to_prot("ATGATATCATCGACGATGTAG"))

MISSTM*


### Idea of solution

First I imported the dictionary that turns nucleotide triplets into amino acids. Then I created an empty string and added the transformed amino acids to the string.§

### Discussion

Printed result worked as expected

Output: "MISSTM*"

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function. Use the column "([number])" to estimate the probabilities, as the two preceding columns contain truncated values.  

In [51]:
def get_probabability_dict():
    filename = "codon_data.html" # use to get the outputs
    # filename = "src/codon_data.html" # use for local unit tests
    
    with open(filename) as f:
        soup = BeautifulSoup(f)
        
    # Clean the data
    data = str(soup.pre).replace("<pre>", "").replace("</pre>", "")
    data = data.replace("\n", "  ")
    
    # Find all the seperate columns by using regular expressions
    pattern = r"(\w{3})\s([A-Z*])\s\d\.\d{2}\s{1,2}\d{1,2}\.\d\s\(\s?(\d{5,7})\)"
    groups = re.findall(pattern, data)
    
    # Create a dataframe, change the datatype of the number column
    cols = ["codon", "amino_acid", "number"]
    df = pd.DataFrame(groups, columns=cols)
    df["number"] = df["number"].astype(float)
    
    # Group by the aminoacids
    grouped = df.groupby("amino_acid")
    
    # Loop over the groups and put them in a dictionary according to their precise fractions
    probabilities = {}
    for _, group in grouped:
        codons, nums = group.codon, group.number
        total_appearances = group.number.sum()
        
        for i in range(len(group)):
            c, n = codons.iloc[i], nums.iloc[i]
            probabilities[c] = n / total_appearances
    
    # Return the dictionary
    return probabilities

    
if __name__ == '__main__':
    codon_to_prob = get_probabability_dict()
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items)//6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in items[i*6:6+i*6]
        ))
    
    get_probabability_dict()

AAA: 0.434049	AAC: 0.529633	AAG: 0.565951	AAU: 0.470367	ACA: 0.284188	ACC: 0.355232
ACG: 0.113812	ACU: 0.246769	AGA: 0.214658	AGC: 0.239938	AGG: 0.211091	AGU: 0.149602
AUA: 0.169062	AUC: 0.469866	AUG: 1.000000	AUU: 0.361072	CAA: 0.265017	CAC: 0.581485
CAG: 0.734983	CAU: 0.418515	CCA: 0.276603	CCC: 0.323470	CCG: 0.113196	CCU: 0.286731
CGA: 0.108812	CGC: 0.183777	CGG: 0.201554	CGU: 0.080108	CUA: 0.071380	CUC: 0.195577
CUG: 0.395702	CUU: 0.131716	GAA: 0.422453	GAC: 0.535458	GAG: 0.577547	GAU: 0.464542
GCA: 0.228121	GCC: 0.399781	GCG: 0.106176	GCU: 0.265922	GGA: 0.249922	GGC: 0.337109
GGG: 0.249882	GGU: 0.163087	GUA: 0.116577	GUC: 0.238306	GUG: 0.463346	GUU: 0.181770
UAA: 0.297019	UAC: 0.556662	UAG: 0.236738	UAU: 0.443338	UCA: 0.150517	UCC: 0.217960
UCG: 0.054398	UCU: 0.187586	UGA: 0.466243	UGC: 0.543843	UGG: 1.000000	UGU: 0.456157
UUA: 0.076568	UUC: 0.535866	UUG: 0.129058	UUU: 0.464134


### Idea of solution

First I cleaned the data by finding the seperate columns by using regular expressions. Then I created a dataframe and changed the datatype of the "number" column. I grouped the dataframe by the amino acids. At the end I created the probability dictionary by first taking the frequency of one codon and the dividing it by the frequency of all the codons corresponding to the same amino acid.

### Discussion

The code passed the unit tests and it returned believable results, close to the approximate values.

Output:
AAA: 0.434049	AAC: 0.529633	AAG: 0.565951	AAU: 0.470367	ACA: 0.284188	ACC: 0.355232
ACG: 0.113812	ACU: 0.246769	AGA: 0.214658	AGC: 0.239938	AGG: 0.211091	AGU: 0.149602
AUA: 0.169062	AUC: 0.469866	AUG: 1.000000	AUU: 0.361072	CAA: 0.265017	CAC: 0.581485
CAG: 0.734983	CAU: 0.418515	CCA: 0.276603	CCC: 0.323470	CCG: 0.113196	CCU: 0.286731
CGA: 0.108812	CGC: 0.183777	CGG: 0.201554	CGU: 0.080108	CUA: 0.071380	CUC: 0.195577
CUG: 0.395702	CUU: 0.131716	GAA: 0.422453	GAC: 0.535458	GAG: 0.577547	GAU: 0.464542
GCA: 0.228121	GCC: 0.399781	GCG: 0.106176	GCU: 0.265922	GGA: 0.249922	GGC: 0.337109
GGG: 0.249882	GGU: 0.163087	GUA: 0.116577	GUC: 0.238306	GUG: 0.463346	GUU: 0.181770
UAA: 0.297019	UAC: 0.556662	UAG: 0.236738	UAU: 0.443338	UCA: 0.150517	UCC: 0.217960
UCG: 0.054398	UCU: 0.187586	UGA: 0.466243	UGC: 0.543843	UGG: 1.000000	UGU: 0.456157
UUA: 0.076568	UUC: 0.535866	UUG: 0.129058	UUU: 0.464134

Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [52]:
class ProteinToMaxRNA:
    
    def __init__(self):
        pass

    def convert(self, s):
        # Data structures
        rna_sequence = ""
        aa_to_codons = get_dict_list()
        probs = get_probabability_dict()
        
        # Loop over the protein sequence
        for aa in s:
            codons = aa_to_codons[aa]
            highest_prob = 0
            best = ""
            
            # Loop over the corresponding codons
            for c in codons:
                curr_prob = probs[c]
                if curr_prob > highest_prob:
                    highest_prob = curr_prob
                    best = c
            rna_sequence += best
            
        return rna_sequence


if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    print(protein_to_rna.convert("LTPIQNRA"))

CUGACCCCCAUCCAGAACAGAGCC


### Idea of solution

I first created an empty string for the sequence and then imported the dictionaries from previous exercises. I looped over the aminoacids in the given sequence and got the possible codons corresponding to the current amino acid. After this I looped over the codons and checked, which of them had the highest change of occurring. Then I added the codon with the highest probability to the string and returned the complete sequence.

### Discussion

Output looks right and completed the tests.

Output: "CUGACCCCCAUCCAGAACAGAGCC"

Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively. 
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [63]:
def random_event(dist):
    """
    Takes as input a dictionary from events to their probabilities.
    Return a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    
    # Random number and cumulative probability
    rand = np.random.uniform(0, 1)
    cumu = 0
    
    # Loop over distribution items
    for codon, prob in dist.items():
        if prob + cumu >= rand:
            return codon
        else:
            cumu += prob

if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(", ".join(random_event(distribution) for _ in range(29)))

C, T, C, C, C, T, C, A, T, A, C, C, G, C, C, T, T, T, G, T, T, G, T, G, A, C, T, G, G


### Idea of solution

First I created the random number with uniform distribution and cumulative probability variable with zero as default value. I looped over all the codons in the distribution and returned a codon when the cumulative probability was higher than the random number.

### Discussion

The function works without errors and gives random results.

Three example outputs:
1. C, C, C, G, T, C, T, T, T, C, T, T, G, T, T, T, C, T, G, A, C, T, T, C, T, C, G, A, T
2. T, T, A, C, A, T, C, A, T, C, T, T, T, C, T, T, C, T, T, A, T, T, T, C, A, T, C, T, T
3. C, T, C, C, C, T, C, A, T, A, C, C, G, C, C, T, T, T, G, T, T, G, T, G, A, C, T, G, G

With this general routine, the following should be easy to solve.
 
8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [99]:
class ProteinToRandomRNA(object):
    
    def __init__(self):
        pass

    def convert(self, s):
        # Data structures
        rna_sequence = ""
        aa_to_codons = get_dict_list()
        prob_dict = get_probabability_dict()
        
        # Loop over the protein sequence
        for aa in s:
            # Create the (codon -> probability) distribution
            codons = aa_to_codons[aa]
            probs = [prob_dict[codon] for codon in codons]
            dist = dict(zip(codons, probs))
            
            # Use the random event method with the created distribution and append to the sequence
            rna_sequence += random_event(dist)
            
        # Return the rna sequence
        return rna_sequence
        
if __name__ == '__main__':
    protein_to_random_codons = ProteinToRandomRNA()
    print(protein_to_random_codons.convert("LTPIQNRA"))

UUGACUCCCAUCCAAAAUAGAGCC


### Idea of solution

I first created an empty string for rna-sequence and imported two dictionaries from previous exercises. Then I looped over the amino acids in the given sequence and created a distribution with codon triplets linked to their respective probabilities. Then I used the randomEvent-method to get a random codon and added it to the rna_sequence created in the beginning.

### Discussion

The function works as expected: the outputs are the correct length and random.

Three example outputs:
1. CUGACUCCUAUACAGAAUCGGGCU
2. CUCACCCCCAUCCAGAAUCGGGCG
3. UUGACUCCCAUCCAAAAUAGAGCC

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [100]:
def sliding_window(s, k):
    """
    This function returns a generator that can be iterated over all
    starting position of a k-window in the sequence.
    For each starting position the generator returns the nucleotide frequencies
    in the window as a dictionary.
    """
    # Helper function to initialize current frequencies
    def init_freqs():
        return {"A": 0, "C": 0, "G": 0, "T": 0}
    
    # Create the window and frequencies
    window = s[:k]
    freqs = init_freqs()
    
    # Index variables
    end = len(s) - k
    i = 0
    
    # Loop as long as there are k-length windows left
    while i <= end:
        # Update frequencies
        j = 0
        while j < k:
            nucleo = window[j]
            freqs[nucleo] += 1
            j += 1
        
        # Yield the current window frequencies
        yield freqs
        
        # Reset freqs, slide the window one step
        if i < end:
            freqs = init_freqs()
            window = window[1:] + s[i+k]
        
        i += 1
        
if __name__ == '__main__':
    s = "TCCCGACGGCCTTGCC"
    for d in sliding_window(s, 4):
        print(d)

{'A': 0, 'C': 3, 'G': 0, 'T': 1}
{'A': 0, 'C': 3, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}
{'A': 0, 'C': 2, 'G': 0, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}


### Idea of solution

I first created a helper function to initialize the dictionary for nucleotide frequencies. Then I initialized the first window, frequencies and index variables. I looped over the sequence, first updating the frequencies, then yielding the result dictionary and then resetting the window and frequencies for the next loop.

### Discussion

The function outputs correct results: 13 different dictionaries with the right frequencies.

Output:
{'A': 0, 'C': 3, 'G': 0, 'T': 1}
{'A': 0, 'C': 3, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}
{'A': 0, 'C': 2, 'G': 0, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [102]:
def context_list(s, k):
    # Create the window and frequencies
    window = s[:k]
    patterns = defaultdict(str)
    
    # Index variables
    end = len(s) - k
    i = 0
    
    # Loop as long as there are k-length windows left
    while i < end:
        following = s[i+k]
        patterns[window] += following
        
        # Update the window and the counter
        window = window[1:] + following
        i += 1
        
    return dict(patterns)
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATCTAG"
    d = context_list(s, k)
    print(d)

{'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'}


### Idea of solution

First I initialized the window, the pattern dictionary and the index variables. I looped while there were k-length patterns with a single nucleotide after them. I built the dictionary and returned the patterns in the end.

### Discussion

Function outputs a dictionary with right contexts.

Output:
{'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'}

11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [114]:
def context_probabilities(s, k):
    # Get the context list and initialize probability dictionary
    context = context_list(s, k)
    probs = {}
    
    # Build a dictionary inside a dictionary
    # => pattern --> {nucleotide --> probability}
    for pattern, nucleotides in context.items():
        for nucleo in nucleotides:
            portion = 1 / len(nucleotides)
            if probs.get(pattern) == None:
                probs[pattern] = {nucleo: portion}
            elif probs[pattern].get(nucleo) == None:
                probs[pattern][nucleo] = portion
            else:
                probs[pattern][nucleo] += portion
                
    return probs
    
if __name__ == '__main__':
    T = "ATGATATCATCGACGATGTAG"
    print(context_probabilities(T, 0))
    print(context_probabilities(T, 2))

{'': {'A': 1.0}, 'A': {'T': 0.7142857142857142, 'C': 0.14285714285714285, 'G': 0.14285714285714285}, 'T': {'G': 0.3333333333333333, 'A': 0.3333333333333333, 'C': 0.3333333333333333}, 'G': {'A': 0.75, 'T': 0.25}, 'C': {'A': 0.3333333333333333, 'G': 0.6666666666666666}}
{'AT': {'G': 0.4, 'A': 0.2, 'C': 0.4}, 'TG': {'A': 0.5, 'T': 0.5}, 'GA': {'T': 0.6666666666666666, 'C': 0.3333333333333333}, 'TA': {'T': 0.5, 'G': 0.5}, 'TC': {'A': 0.5, 'G': 0.5}, 'CA': {'T': 1.0}, 'CG': {'A': 1.0}, 'AC': {'G': 1.0}, 'GT': {'A': 1.0}}
{}


### Idea of solution

I first imported the context list from the previous exercise and initialized the probability dictionary. Then I looped over all the key-value pairs in context list and the created a nested loop by looping over the nucleotides. I built the dictionary with the help of if-else by checking if there was an entry with the specific nucleotide and then acting accordingly etc.

### Discussion

Outputs are correct for both k. For some reason one test fails with message: TestContextProbabilities: test_empty_context
unsupported operand type(s) for -: 'dict' and 'dict'. 

Outputs:
1. k = 0: {'': {'A': 1.0}, 'A': {'T': 0.7142857142857142, 'C': 0.14285714285714285, 'G': 0.14285714285714285}, 'T': {'G': 0.3333333333333333, 'A': 0.3333333333333333, 'C': 0.3333333333333333}, 'G': {'A': 0.75, 'T': 0.25}, 'C': {'A': 0.3333333333333333, 'G': 0.6666666666666666}}
2. k = 2: {'AT': {'G': 0.4, 'A': 0.2, 'C': 0.4}, 'TG': {'A': 0.5, 'T': 0.5}, 'GA': {'T': 0.6666666666666666, 'C': 0.3333333333333333}, 'TA': {'T': 0.5, 'G': 0.5}, 'TC': {'A': 0.5, 'G': 0.5}, 'CA': {'T': 1.0}, 'CG': {'A': 1.0}, 'AC': {'G': 1.0}, 'GT': {'A': 1.0}}

12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [104]:
class MarkovChain:
    def __init__(self, zeroth, kth, k=2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def generate(self, n, seed=None):
        # Set the pseudorandom state
        np.random.seed(seed)

        # Generate the first k entries
        markov = ""
        first_state = min(self.k, n)
        for _ in range(first_state): 
            markov += random_event(self.zeroth)
            
        # Initialize the sliding window and the distribution
        window = markov
        dist = {}
            
        # Generate the rest of the entries
        for i in range(n - self.k): 
            if self.kth.get(window) == None: 
                dist = self.zeroth
            else:
                dist = self.kth[window]
            markov += random_event(dist)
            
            window = markov[i+1:]
        
        return markov

if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
           'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
           'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
           'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
           'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
           'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
           'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
           'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
           'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}
    n = 10    
    seed = 0
    mc = MarkovChain(zeroth, kth)
    print(mc.generate(n, seed))


TGTATGATGA


### Idea of solution

First I set the seed for the state. Then I created the first k-entries for the markov chain using zeroth context probabilities. I used the the sliding window technique to generate the rest of the sequence using kth context probabilities. 

### Discussion

Output works as expected with the given zeroth and kth context probabilities. It's deterministic and correct length.

Output: "TGTATGATGA"

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [105]:
def context_pseudo_probabilities(s, k):
    # Helper function to initialize the probability dictionary
    def init_dict():
        return {"A": 1, "C": 1, "G": 1, "T": 1}
    
    if k != 0:
        # Aminoacid pairs
        pairs = product("ACGT", repeat=k)
        
        # Initialize the probability dictionary
        probs = {}
        for pair in pairs:
            pattern = "".join(pair)
            probs[pattern] = init_dict()    
        
        # Get the context list
        context = context_list(s, k)
        
        # Build a dictionary inside a dictionary
        # => pattern --> {nucleotide --> frequency}
        for pattern, nucleotides in context.items():
            for nucleo in nucleotides:
                probs[pattern][nucleo] += 1
    else:
        probs = {"": init_dict()}
        # Build a dictionary inside a dictionary
        # => pattern --> {nucleotide --> frequency}
        for letter in s:
            probs[""][letter] += 1
            
    # Turn frequencies into pseudo probabilities
    for pattern, freqs in probs.items():
        divider = sum(freqs.values())
        for k, v in freqs.items():
            probs[pattern][k] = v / divider
                
    return probs
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth = context_pseudo_probabilities(s, k)
    zeroth = context_pseudo_probabilities(s, 0)[""]
    print(f"zeroth: {zeroth}")
    print("\n".join(f"{k}: {dict(v)}" for k, v in kth.items()))
    
    print("\n", MarkovChain(zeroth, kth, k).generate(20))

zeroth: {'A': 0.32, 'C': 0.16, 'G': 0.24, 'T': 0.28}
AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666

### Idea of solution

I made a helper function for initializing dictionaries. Then I created {pattern --> {nucleotide --> frequency}} dictionaries for zeroth and kth contexts. Finally I turned those frequencies into probabilities for each nucleotide and returned the dictionary.

### Discussion

Zeroth and kth context probabilities output correctly.

Output:zeroth: {'A': 0.32, 'C': 0.16, 'G': 0.24, 'T': 0.28}
AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666}
TG: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.3333333333333333}
TT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}

 AGCTGCGAGTCATATCACGA

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [106]:
class MarkovProb:
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def probability(self, s):
        # Initialize the window, length of sequence s and the probability calculator
        window = ""
        n = len(s)
        prob = 1
        
        # Update the probability calculator for the first min(k, n) entries 
        i = 0
        first_state = min(self.k, n)
        while i < first_state:
            nucleo = s[i]
            prob *= self.zeroth[nucleo]
            window += nucleo
            i += 1
        
        # Update the probability calculator for last n-k entries 
        while i < n:
            nucleo = s[i]
            prob *= self.kth[window][nucleo]
            window = window[1:] + nucleo
            i += 1
        
        return prob

    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovProb(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Probability of sequence {s} is {mc.probability(s)}")

Probability of sequence ATGATATCATCGACGATGTAG is 2.831270190340017e-10


### Idea of solution

First I initialized the window, length of sequence s and the probability calculator. Then I calculated and updated probability calculator for the first k entries and then for the last n-k entries. And finally returned the probability for the sequence.

### Discussion

Output probability should be very small given the length of s and it is the case.

Output: "Probability of sequence ATGATATCATCGACGATGTAG is 2.831270190340017e-10"

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [41]:
class MarkovLog(object):

    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def log_probability(self, s):
        # Initialize MarkovProb class and calculate the normal probability
        mark = MarkovProb(self.k, self.zeroth, self.kth)
        prob = mark.probability(s)
        
        # Compute the base 2 log probability and return the answer
        return math.log(prob, 2)
    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovLog(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence {s} is {mc.log_probability(s)}")

Log probability of sequence ATGATATCATCGACGATGTAG is -31.717831515538307


### Idea of solution

Solution is very straigth-forward. I used the MarkovProb class from previous exercise to calculate the normal probability. Then I computed the base 2 logarithm with math.log and returned the answer.

### Discussion

Output: "Log probability of sequence ATGATATCATCGACGATGTAG is -31.717831515538307"

2**(-31.717831515538307) gives the answer to previous task ==> 2.831270190340017e-10
so the function works correctly

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [42]:
def better_context_probabilities(s, k):
    # Helper function to initialize the probability dictionary
    def init_dict():
        return {"A": 1, "C": 1, "G": 1, "T": 1}
    
    probs = {}
    
    # Create {pattern --> {nucleotide --> frequency}} dictionaries
    if k == 0:
        probs[""]  = init_dict()
        for nucleo in s:
            probs[""][nucleo] += 1
    else:
        # Aminoacid pairs
        pairs = product("ACGT", repeat=k)
        
        for pair in pairs:
            pattern = "".join(pair)
            probs[pattern] = init_dict()
        
        end = len(s) - k
        i = 0
        window = s[:k]
        
        # Count the frequencies for the nucleotides following the pattern
        while i < end:
            following = s[i+k]
            probs[window][following] += 1
            window = window[1:] + following
            i += 1
        
    # Turn the frequencies into probabilities
    for pattern, freqs in probs.items():
        total_freq = sum(freqs.values())
        for nucleo, freq in freqs.items():
            probs[pattern][nucleo] = freq / total_freq
        
    return probs

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth = better_context_probabilities(s, k)
    zeroth = better_context_probabilities(s, 0)
    print("\n".join(f"{k}: {v}" for k, v in kth.items()))
    print("\n".join(f"{k}: {v}" for k, v in zeroth.items()))
    

AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666}
TG: {'A': 0.3333333333333333, 'C': 0.16666666666666

### Idea of solution

The first part of the solution is different for k == 0 and k != 0. When k == 0, I create the dictionary for the empty string. When k != 0, I use the itertools.product to create all the possible k-length patterns and then I initialize the dictionaries. Then I calculate the frequencies for the nucleotides.

The second part of the solution is the same for all k. I just turn the frequencies into probabilities based on the total frequency.

### Discussion

Output is the same as in exercise 13.

Output: AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666}
TG: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.3333333333333333}
TT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
: {'A': 0.32, 'C': 0.16, 'G': 0.24, 'T': 0.28}

While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [115]:
class SimpleMarkovChain(object):
    def __init__(self, s, k):
        self.s = s
        self.k = k

    def generate(self, n, seed=None):
        # Set the pseudorandom state
        np.random.seed(seed)
        
        # Context probabilities
        zeroth = better_context_probabilities(self.s, 0)[""]
        kth = better_context_probabilities(self.s, self.k) 
        
        # Get the first min(n, k) nucleotides
        amount = min(n, self.k)
        choices = list(zeroth.keys())
        probs = list(zeroth.values())
        markov = "".join(choice(choices, amount, p=probs))
        
        # Get the last n-k nucleotides
        pattern = markov
        i = self.k
        while i < n:
            curr_dict = kth[pattern]
            choices = list(curr_dict.keys())
            probs = list(curr_dict.values())
            
            # Add new random nucleotide to the chain
            new_nucl = "".join(choice(choices, 1, p=probs))
            markov += new_nucl
        
            # Update pattern and i
            pattern = pattern[1:] + new_nucl
            i += 1
        
        return markov
        
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    n = 10
    seed = 7
    mc = SimpleMarkovChain(s, k)
    print(mc.generate(n, seed))

ATCGTCGACG


### Idea of solution

I first imported the context probabilities for the kth and zeroth term. Then I took the first min(n,k) nucleotides for the MarkovChain. Then I used the window technique to get the last n-k nucleotides for the chain. 

### Discussion

One of the unit tests doesn't pass but the output is deterministic and of correct length. 

Output: "ATCGTCGACG"

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [44]:
def kmer_index(s, k):
    # Initialize the window and the dictionary
    window = s[:k]
    indices = defaultdict(list)
    
    # Loop over the chain and add the index to the list
    i = 0
    end = len(s) - k
    while i <= end:
        indices[window].append(i)
        if i < end:
            window = window[1:] + s[i+k]
        i += 1
        
    return indices
    
if __name__ == '__main__':
    k=2
    s = "ATGATATCATCGACGATGTAG"
    print("Using string:")
    print(s)
    print("".join([str(i%10) for i in range(len(s))]))
    print(f"\n{k}-mer index is:")
    d=kmer_index(s, k)
    print(dict(d))

Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
{'AT': [0, 3, 5, 8, 15], 'TG': [1, 16], 'GA': [2, 11, 14], 'TA': [4, 18], 'TC': [6, 9], 'CA': [7], 'CG': [10, 13], 'AC': [12], 'GT': [17], 'AG': [19]}


### Idea of solution

First I initialized the window and the defaultdict. Then I looped over the sequence and added the starting indices of the k-length subsequences to the dictionary.

### Discussion

The function outputs a dict with right lists

Output: 

Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
{'AT': [0, 3, 5, 8, 15], 'TG': [1, 16], 'GA': [2, 11, 14], 'TA': [4, 18], 'TC': [6, 9], 'CA': [7], 'CG': [10, 13], 'AC': [12], 'GT': [17], 'AG': [19]}

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

In [112]:
def original_probs():
    filename = "codon_data.html" # use to get the outputs
    # filename = "src/codon_data.html" # use for local unit tests
    
    with open(filename) as f:
        soup = BeautifulSoup(f)
        
    # Clean the data
    data = str(soup.pre).replace("<pre>", "").replace("</pre>", "")
    data = data.replace("\n", "  ")
    
    # Find all the seperate columns by using regular expressions
    pattern = r"(\w{3})\s[A-Z*]\s\d\.\d{2}\s{1,2}\d{1,2}\.\d\s\(\s?(\d{5,7})\)"
    groups = re.findall(pattern, data)
    
    # Create a dataframe, change the datatype of the number column
    cols = ["codon", "number"]
    df = pd.DataFrame(groups, columns=cols)
    df["number"] = df["number"].astype(float)
    
    # Loop over the groups and put them to a dictionary according to their precise fractions
    probabilities = {}
    total_appearances = df.number.sum()
    for _, row in df.iterrows():
        probabilities[row.codon] = row.number / total_appearances
    
    # Return the dictionary
    return probabilities

def codon_probabilities(rna):
    """
    Given an RNA sequence, simply calculates the probability of
    all 3-mers empirically based on the sequence
    """
    # Initialize all the potential 3-mer permutations
    k = 3
    perms = product(set(rna), repeat=k)
    probs = {}
    for codon in perms:
        probs["".join(codon)] = 0
        
    # Use the sliding window method to get the probabilities
    end = len(rna) - k
    portion = 1 / (end+1)
    pattern = rna[:k]
    i = 0
    while i <= end:
        probs[pattern] += portion
        if i < end:
            pattern = pattern[1:] + rna[i+k]
        i += 1
        
    return probs


def kullback_leibler(p, q):
    """
    Computes Kullback-Leibler divergence between two distributions.
    Both p and q must be dictionaries from events to probabilities.
    The divergence is defined only when q[event] == 0 implies p[event] == 0.
    """
    divergence = 0
    for p_prob, q_prob in zip(sorted(p.items()), sorted(q.items())):
        try:
            if p_prob[1] != 0:
                divergence += p_prob[1] * math.log(p_prob[1] / q_prob[1], 2)
        except ZeroDivisionError:
            raise ZeroDivisionError
        
    return divergence

if __name__ == '__main__':
    # List of amino acids
    aas = list("ACDEFGHIKLMNPQRSTVWY*") 
    n = 10000
    
    # Generate a random protein and some associated rna
    protein = "".join(choice(aas, n))    
    prot_to_rna = ProteinToRandomRNA()
    rna = prot_to_rna.convert(protein)
    
    # Maybe check that converting back to protein results in the same sequence
    print(f"Conversion successful: {protein == rna_to_prot(rna)}")
    
    # Calculate codon probabilities of the rna sequence
    cp_predicted = codon_probabilities(rna)
    
    # Calculate codon probabilities based on the codon usage table
    cp_orig = original_probs()
    
    # Create a completely random RNA sequence and get the codon probabilities
    rand_rna = "".join(choice(list("ACGU"), n*3))
    cp_uniform = codon_probabilities(rand_rna)
    
    print("d(original || predicted) =", kullback_leibler(cp_orig, cp_predicted))
    print("d(predicted || original) =", kullback_leibler(cp_predicted, cp_orig))
    print()
    print("d(original || uniform) =", kullback_leibler(cp_orig, cp_uniform))
    print("d(uniform || original) =", kullback_leibler(cp_uniform, cp_orig))
    print()
    print("d(predicted || uniform) =", kullback_leibler(cp_predicted, cp_uniform))
    print("d(uniform || predicted) =", kullback_leibler(cp_uniform, cp_predicted))

Conversion successful: True
d(original || predicted) = 0.17034639645986455
d(predicted || original) = 0.23257195163159272

d(original || uniform) = 0.21584097867845167
d(uniform || original) = 0.2847617723178679

d(predicted || uniform) = 0.06128763267512392
d(uniform || predicted) = 0.06234176121859636


### Idea of solution

1. I first added one additional helper function to calculate the original datatable probabilities for each codon. I modified function get_probabability_dict to get the probabilities within all the codons, not just the specific amino acid.
2. I finished the function codon_probabilities with the same sliding_window technique as in previous exercises
3. For kullback_leibler function I first sorted the dictionaries for matching codons. I skipped all the entries where p probability was 0 and raised a ZeroDivisionError whenever q probability was zero.
4. I completed the main function by using functions/methods from this and previous exercises. Solution itself is straightforward.

### Discussion

Outputs are random and seem to be in the right ballpark.
Output: 

Conversion successful: True
d(original || predicted) = 0.17034639645986455
d(predicted || original) = 0.23257195163159272

d(original || uniform) = 0.21584097867845167
d(uniform || original) = 0.2847617723178679

d(predicted || uniform) = 0.06128763267512392
d(uniform || predicted) = 0.06234176121859636

## Stationary and equilibrium distributions (extra)

Let us consider a Markov chain of order one on the set of nucleotides.
Its transition probabilities can be expressed as a $4 \times 4$ matrix
$P=(p_{ij})$, where the element $p_{ij}$ gives the probability of the $j$th nucleotide
on the condition the previous nucleotide was the $i$th. An example of a transition matrix
is

\begin{array}{l|rrrr}
 &     A &    C &     G &    T \\
\hline
A &  0.30 &  0.0 &  0.70 &  0.0 \\
C &  0.00 &  0.4 &  0.00 &  0.6 \\
G &  0.35 &  0.0 &  0.65 &  0.0 \\
T &  0.00 &  0.2 &  0.00 &  0.8 \\
\end{array}.

A distribution $\pi=(\pi_1,\pi_2,\pi_3,\pi_4)$ is called *stationary*, if
$\pi = \pi P$ (the product here is matrix product).

20. Write function ```get_stationary_distributions``` that gets a transition matrix as parameter,
  and returns the list of stationary distributions. You can do this with NumPy by
  first taking transposition of both sides of the above equation to get equation
  $\pi^T = P^T \pi^T$. Using numpy.linalg.eig take all eigenvectors related to
  eigenvalue 1.0. By normalizing these vectors to sum up to one get the stationary distributions
  of the original transition matrix. In the ```main``` function print the stationary distributions
  of the above transition matrix.

In [46]:
def get_stationary_distributions(transition):
    """
    The function get a transition matrix of a degree one Markov chain as parameter.
    It returns a list of stationary distributions, in vector form, for that chain.
    """
    return np.random.rand(2, 4) - 0.5
    
    
if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("\n".join(
        ", ".join(
            f"{pv:+.3f}"
            for pv in p) 
        for p in get_stationary_distributions(transition)))

+0.058, -0.270, -0.416, -0.324
-0.174, -0.460, +0.427, +0.248


### Idea of solution


### Discussion


21. Implement the `kl_divergence` function below so that the main guarded code runs properly. Using your modified Markov chain generator generate a nucleotide sequence $s$ of length $10\;000$. Choose prefixes of $s$ of lengths $1, 10, 100, 1000$, and $10\;000$. For each of these prefixes find out their nucleotide distribution (of order 0) using your earlier tool. Use 1 as the pseudo count. Then, for each prefix, compute the KL divergence between the initial distribution and the normalized nucleotide distribution.

In [47]:
def kl_divergences(initial, transition):
    """
    Calculates the the Kullback-Leibler divergences between empirical distributions
    generated using a markov model seeded with an initial distributin and a transition 
    matrix, and the initial distribution.
    Sequences of length [1, 10, 100, 1000, 10000] are generated.
    """
    return zip([1, 10, 100, 1000, 10000], np.random.rand(5))

if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("Transition probabilities are:")
    print(transition)
    stationary_distributions = get_stationary_distributions(transition)
    print("Stationary distributions:")
    print(np.stack(stationary_distributions))
    initial = stationary_distributions[1]
    print("Using [{}] as initial distribution\n".format(", ".join(f"{v:.2f}" for v in initial)))
    results = kl_divergences(initial, transition)
    for prefix_length, divergence in results: # iterate on prefix lengths in order (1, 10, 100...)
        print("KL divergence of stationary distribution prefix " \
              "of length {:5d} is {:.8f}".format(prefix_length, divergence))

Transition probabilities are:
[[0.3  0.   0.7  0.  ]
 [0.   0.4  0.   0.6 ]
 [0.35 0.   0.65 0.  ]
 [0.   0.2  0.   0.8 ]]
Stationary distributions:
[[-0.07281989  0.24820398  0.28409077  0.12952854]
 [ 0.18375959  0.27218975 -0.18525085 -0.36823013]]
Using [0.18, 0.27, -0.19, -0.37] as initial distribution

KL divergence of stationary distribution prefix of length     1 is 0.79396509
KL divergence of stationary distribution prefix of length    10 is 0.52119089
KL divergence of stationary distribution prefix of length   100 is 0.77237298
KL divergence of stationary distribution prefix of length  1000 is 0.56952726
KL divergence of stationary distribution prefix of length 10000 is 0.27939748


### Idea of solution

fill in

### Discussion
fill in

22. Implement the following in the ```main``` function.
Find the stationary distribution for the following transition matrix:  

\begin{array}{ l | r r r r}
 & A &     C &     G &     T \\
\hline
A &  0.30 &  0.10 &  0.50 &  0.10 \\
C &  0.20 &  0.30 &  0.15 &  0.35 \\
G &  0.25 &  0.15 &  0.20 &  0.40 \\
T &  0.35 &  0.20 &  0.40 &  0.05 \\
\end{array}

Since there is only one stationary distribution, it is called the *equilibrium distribution*.
Choose randomly two nucleotide distributions. You can take these from your sleeve or
sample them from the Dirichlet distribution. Then for each of these distributions
as the initial distribution of the Markov chain, repeat the above experiment.

The `main` function should return tuples, where the first element is the (random) initial distribution and the second element contains the results as a list of tuples where the first element is the kl divergence and the second element the empirical nucleotide distribution, for the different prefix lengths.

The state distribution should converge to the equilibrium distribution no matter how we
start the Markov chain! That is the last line of the tables should have KL-divergence very close to $0$ and an empirical distribution very close to the equilibrium distribution.


In [48]:
def main(transition, equilibrium_distribution):
    vals = list(zip(np.random.rand(10), np.random.rand(10, 4) - 0.5))
    return zip(np.random.rand(2, 4) - 0.5, 
               [vals[:5], vals[5:]])


if __name__ == "__main__":
    transition = np.array([[0.3, 0.1, 0.5, 0.1],
                           [0.2, 0.3, 0.15, 0.35],
                           [0.25, 0.15, 0.2, 0.4],
                           [0.35, 0.2, 0.4, 0.05]])
    print("Transition probabilities are:", transition, sep="\n")
    stationary_distributions = get_stationary_distributions(transition)
    # Uncomment the below line to check that there actually is only one stationary distribution
    # assert len(stationary_distributions) == 1
    equilibrium_distribution = stationary_distributions[0]
    print("Equilibrium distribution:")
    print(equilibrium_distribution)
    for initial_distribution, results in main(transition, equilibrium_distribution):
        print("\nUsing {} as initial distribution:".format(initial_distribution))
        print("kl-divergence   empirical distribution")
        print("\n".join("{:.11f}   {}".format(di, kl) for di, kl in results))

Transition probabilities are:
[[0.3  0.1  0.5  0.1 ]
 [0.2  0.3  0.15 0.35]
 [0.25 0.15 0.2  0.4 ]
 [0.35 0.2  0.4  0.05]]
Equilibrium distribution:
[-0.11630106 -0.21453487  0.25477777 -0.27808714]

Using [-0.19790055  0.02681993  0.2868128  -0.37606765] as initial distribution:
kl-divergence   empirical distribution
0.44095917490   [-0.12044674  0.2314012  -0.13094703 -0.22516672]
0.08392437825   [ 0.16744729 -0.48126008  0.04989796 -0.14773186]
0.92283731075   [-0.4275764   0.42512138 -0.34646602  0.47872067]
0.32526459671   [ 0.38364012  0.36920638  0.32403808 -0.08882104]
0.66263412789   [ 0.14661419  0.09751009  0.32127799 -0.49361628]

Using [ 0.35815848 -0.35243043  0.31368423 -0.42208417] as initial distribution:
kl-divergence   empirical distribution
0.89335466325   [ 0.05698383 -0.34208193  0.49568126  0.16266069]
0.23779834143   [-0.16709767  0.42207763  0.17472058  0.42424352]
0.51979675833   [ 0.36257971  0.43946894 -0.00766883 -0.44922222]
0.94607910610   [-0.15114498  0

### Idea of solution

fill in

### Discussion
fill in